# Duration of Sick Notes and Associated Diagnoses

This notebook provides descriptive statistics (mean and median) of the duration of sick notes and compares diagnoses associated with sick notes across COVID and comparator cohorts.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns

from functools import reduce
from glob import glob

pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth',1000)

In [ ]:
# Function to parse string
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

def round_seven(x, base=7):
    return base * round(x/base)

In [ ]:
# Read in and append input files
li = []

for file in glob('../output/cohorts/cohort_rates*.dta'):
    df_temp = pd.read_stata(file)
    # Creates date variable based on file name
    df_temp['cohort'] = file[find_nth(file, '_', 2)+1:-4]
    # Create population count
    df_temp['population'] = 1
    li.append(df_temp)
    
df_temp = pd.concat(li, axis=0, ignore_index=False).reset_index(drop=True)

# hospitalised COVID cohorts
covid_hosp_2020 = df_temp.loc[(~df_temp.hosp_expo_date.isna()) & (df_temp.cohort == 'covid_2020')]
covid_hosp_2020.cohort = 'covid_hosp_2020'
li.append(covid_hosp_2020)

covid_hosp_2021 = df_temp.loc[(~df_temp.hosp_expo_date.isna()) & (df_temp.cohort == 'covid_2021')]
covid_hosp_2021.cohort = 'covid_hosp_2021'
li.append(covid_hosp_2021)

df_input = pd.concat(li, axis=0, ignore_index=False).reset_index(drop=True)

In [ ]:
# Do not count those who had sick notes beyond end dates
df_input.loc[df_input.sick_note == 0, 'first_sick_note_duration'] = np.nan

In [ ]:
# Columns to subset
diag_cols = [col for col in df_input if col.startswith('diag_')]
subset_cols = ['cohort','age_group','sex',
               'ethnicity','imd','region_string',
               'first_sick_note_duration',
               'sick_note','population'] + diag_cols

# Subset to relevant columns and do not count those beyond 
df_clean = df_input[subset_cols].rename(columns={'region_string':'region'})

In [ ]:
# Clean cohort names
dict_cohort_names = {'covid_2020':'COVID patients (2020)', 
                     'covid_2021':'COVID patients (2021)', 
                     'covid_hosp_2020':'Hospitalised COVID patients (2020)', 
                     'covid_hosp_2021':'Hospitalised COVID patients (2021)', 
                     'general_2019':'General population (2019)', 
                     'general_2020':'General population (2020)', 
                     'general_2021':'General population (2021)', 
                     'pneumonia_2019':'Hospitalised pneumonia patients (2019)'}
df_clean = df_clean.replace({"cohort": dict_cohort_names})

_____

## Duration

In [ ]:
def compute_med_mean(path, demo=''):
    groups = ['cohort']
    if demo != '': 
        groups = ['cohort', demo]
    df_pct_ct = df_clean.groupby(
        groups)[['sick_note','population']].sum().reset_index()
    df_med = df_clean.groupby(
        groups)[['first_sick_note_duration']].apply(np.nanmedian).reset_index().rename(columns={0:'median_duration'})
    df_mean = df_clean.groupby(
        groups)[['first_sick_note_duration']].apply(np.nanmean).reset_index().rename(columns={0:'mean_duration'})
    df_pct25 = df_clean.groupby(
        groups)[['first_sick_note_duration']].apply(lambda x: np.nanpercentile(x,25)).reset_index().rename(columns={0:'pct_25'})
    df_pct75 = df_clean.groupby(
        groups)[['first_sick_note_duration']].apply(lambda x: np.nanpercentile(x,75)).reset_index().rename(columns={0:'pct_75'})
    dfs = [df_pct_ct, df_med, df_mean, df_pct25, df_pct75]
    df_out = reduce(
        lambda left,right: pd.merge(left,right,on=groups), dfs
    )
    # Redact if sick notes or population count <= 5 
    df_out = df_out.loc[(df_out['population'] > 5) & (df_out['sick_note'] > 5)]
    df_out['population'] = df_out['population'].apply(lambda x: round_seven(x))
    df_out['sick_note'] = df_out['sick_note'].apply(lambda x: round_seven(x))
    Path("../output/tabfig/").mkdir(parents=True, exist_ok=True)
    df_out.to_csv('../output/tabfig/' + path + '.csv', index=False)
    return df_out

In [ ]:
# Overall
compute_med_mean('med_mean_overall')

In [ ]:
# Age group
compute_med_mean('med_mean_age_group', 'age_group')

In [ ]:
# Sex
compute_med_mean('med_mean_sex', 'sex')

In [ ]:
# Ethnicity
compute_med_mean('med_mean_ethnicity', 'ethnicity')

In [ ]:
# IMD
compute_med_mean('med_mean_imd', 'imd')

In [ ]:
# Region
compute_med_mean('med_mean_region', 'region')

_______

## Associated Diagnoses

### Percentage of Sick Notes by Diagnostic Category

In [ ]:
def create_subplot(measure, df_in, ax):
    # Create barplot
    sns.barplot(x=measure, y='cohort', data=df_in, ax=ax, orient='h')
    # Set title and axes labels
    ax.set_xlabel('% of Sick Notes')
    ax.set_ylabel('Cohort')
    
def create_plotgrid(measure, df_in):
    fig, ax = plt.subplots(figsize=(12,8))
    create_subplot('pct_'+measure, df_in, ax)
    # Title
    title = measure[5:].replace("_", " ").title()
    ax.title.set_text(title) 
    plt.show()
    Path("../output/tabfig/").mkdir(parents=True, exist_ok=True)
    fig.savefig('../output/tabfig/' + measure + '.png')

In [ ]:
# Create flag
for c in diag_cols:
    df_clean['num_' + c] = 0
    df_clean.loc[~df_clean[c].isna(), 'num_' + c] = 1

# Sum sick notes and diagnoses by cohort
num_diag_cols = [col for col in df_clean if col.startswith('num_diag_')]
diag_subset_cols = ['cohort', 'sick_note'] + num_diag_cols
df_diag = df_clean[diag_subset_cols]
df_diag_sum = df_diag.groupby('cohort').sum().reset_index()

# Proportion of sick notes by diagnoses
for c in num_diag_cols:
    df_diag_sum['pct_diag_' + c[9:]] = (df_diag_sum[c]/df_diag_sum['sick_note'])*100
pct_cols = [col for col in df_diag_sum if col.startswith('pct_diag_')]

In [ ]:
for c in diag_cols:
    # Subset and redact if <= 5 in sick note or diagnosis count
    df_subset = df_diag_sum[['cohort', 'sick_note', 'num_'+c, 'pct_'+c]]
    df_subset.loc[(df_subset['sick_note'] < 5) | (df_subset['num_'+c] < 5), ('num_'+c, 'pct_'+c)] = (0,0)
    try:
        create_plotgrid(c, df_subset)
    except ValueError:
        pass

### Top Codes by Category

In [ ]:
# Map categories to codelists
dict_cat_codes = {
    "diag_central_nervous_system": 'central-nervous-system-finding',
    "diag_pregnancy_complication": 'complication-of-pregnancy-childbirth-andor-the-puerperium',
    "diag_congenital_disease": 'congenital-disease',
    "diag_auditory_disorder": 'disorder-of-auditory-system',
    "diag_cardio_disorder": 'disorder-of-cardiovascular-system',
    "diag_bloodcell_disorder": 'disorder-of-cellular-component-of-blood',
    "diag_connective_tissue": 'disorder-of-connective-tissue',
    "diag_digestive_disorder": 'disorder-of-digestive-system',
    "diag_endocrine_disorder": 'disorder-of-endocrine-system',
    "diag_fetus_newborn_disorder": 'disorder-of-fetus-or-newborn',
    "diag_hematopoietic_disorder": 'disorder-of-hematopoietic-structure',
    "diag_immune_disorder": 'disorder-of-immune-function',
    "diag_labor_delivery_disorder": 'disorder-of-labor-delivery',
    "diag_musculoskeletal_disorder": 'disorder-of-musculoskeletal-system',
    "diag_nervous_disorder": 'disorder-of-nervous-system',
    "diag_puerperium_disorder": 'disorder-of-puerperium',
    "diag_respiratory_disorder": 'disorder-of-respiratory-system',
    "diag_skin_disorder": 'disorder-of-skin-andor-subcutaneous-tissue',
    "diag_genitourinary_disorder": 'disorder-of-the-genitourinary-system',
    "diag_infectious_disease": 'infectious-disease',
    "diag_mental_disorder": 'mental-disorder',
    "diag_metabolic_disease": 'metabolic-disease',
    "diag_neoplastic_disease": 'neoplastic-disease',
    "diag_nutritional_disorder": 'nutritional-disorder',
    "diag_poisoning": 'poisoning',
    "diag_trauma": 'traumatic-andor-non-traumatic-injury',
    "diag_visual_disorder": 'visual-system-disorder',
}

In [ ]:
# Import codelist to get codenames
def import_codelist(measure):
    fname = dict_cat_codes.get(measure)
    df_codelist = pd.read_csv(f'../codelists/user-kate-mansfield-{fname}-all-descendants.csv')
    return df_codelist

def top_codes(measure):
    # Sum by code
    df_cln_codes = df_clean[['cohort',measure,'population']].rename(columns={'population':'count',measure:'code'})
    df_cln_codes_agg = df_cln_codes.groupby(['cohort','code']).sum().reset_index()

    # Add code names
    df_codelist = import_codelist(measure)
    df_code_sum = df_cln_codes_agg.merge(df_codelist, on='code', how='left')

    # Redact if count <= 5 and limit to top 5 by cohort
    df_code_sum = df_code_sum.loc[df_code_sum['count'] > 5]
    df_code_sum = df_code_sum[['cohort','code','term','count']].reset_index(drop=True)
    df_code_sum = df_code_sum.sort_values(['cohort','count'],ascending=False).groupby(['cohort'],sort=False).head(5).reset_index(drop=True)
    
    # Round to nearest 5 
    df_code_sum['count'] = df_code_sum['count'].apply(lambda x: round_seven(x))
    df_code_sum['code'] = df_code_sum['code'].apply(lambda x: str(int(x)))

    # Display
    title = measure[5:].replace("_", " ").title()
    print(f'Top Codes ({title})')
    display(df_code_sum)
    print('\n')

In [ ]:
for c in diag_cols:
    top_codes(c)

### Percentage of Sick Notes Across All Diagnostic Categories

In [ ]:
# Limit columns
df_sick_notes = df_clean[['cohort','sick_note'] + diag_cols]

# Reshape long
df_sick_notes_long = pd.melt(df_sick_notes, id_vars=['cohort','sick_note'], value_vars=diag_cols).drop(columns=['variable'])
df_sick_notes_long = df_sick_notes_long.loc[~df_sick_notes_long.value.isna()].reset_index(drop=True)

# Create sums and percentages
df_code_sum = df_sick_notes_long.groupby(['cohort','value']).sum().reset_index()
df_cohort_sum = df_code_sum[['cohort','sick_note']].groupby(['cohort']).sum().reset_index().rename(columns={'sick_note':'total_sick_note'})
df_all_diag = df_code_sum.merge(df_cohort_sum, on='cohort', how='left')
df_all_diag['pct_diag_all_codes'] = (df_all_diag['sick_note']/df_all_diag['total_sick_note'])*100

In [ ]:
create_plotgrid('diag_all_codes',df_all_diag)

### Top Codes by All Diagnostic Categories

In [ ]:
# Import and append codelists
li_diag_cl = []

for k in dict_cat_codes:
    df_temp = import_codelist(k)
    li_diag_cl.append(df_temp)
    
df_codelists = pd.concat(li_diag_cl, ignore_index=True).drop_duplicates().reset_index(drop=True)

In [ ]:
# Merge code names
df_all_code = df_all_diag[['cohort','value','sick_note']].rename(columns={'value':'code','sick_note':'count'})
df_all_code_names = df_all_code.merge(df_codelists, on='code', how='left')[['cohort','code','term','count']]

# Restrict to top 5 (redact if count <= 5)
df_all_code_names = df_all_code_names.loc[df_all_code_names['count'] > 5]
df_all_code_names = df_all_code_names.sort_values(['cohort','count'],ascending=False).groupby(['cohort'],sort=False).head(5).reset_index(drop=True)

# Round to nearest 5 
df_all_code_names['count'] = df_all_code_names['count'].apply(lambda x: round_seven(x))
df_all_code_names['code'] = df_all_code_names['code'].apply(lambda x: str(int(x)))

In [ ]:
print('Top Codes')
display(df_all_code_names)